In [2]:
import openrouteservice as ors
import folium

client = ors.Client(key='5b3ce3597851110001cf6248f4b857eea0554d93886b4c09e748b2bf')

In [86]:
import re
import pandas as pd
import math

import numpy as np

from operator import itemgetter
from heapq import nsmallest

from itertools import chain

In [134]:
df_property = pd.read_csv('../data/curated/property.csv')

property_coor = []
for index, row in df_property.iterrows():
    if re.search(r'\[(-\d+.\d+)',str(row['coordinates'])): 
        lat = float(re.findall(r'\[(-\d+.\d+)',str(row['coordinates']))[0])
        long = float(re.findall(r'(\d+.\d+)\]',str(row['coordinates']))[0])
        property_coor.append((long,lat))
    else:
        property_coor.append((0,0))
print(len(property_coor))

df_school = pd.read_csv('../data/curated/school.csv')

school_coor = []
for index, row in df_school.iterrows():
    if not math.isnan(row['Y']):
        lat = row['Y']
        long = row['X']
        school_coor.append((long,lat))
    else:
        school_coor.append((0,0))
print(len(school_coor))


15226
2299


In [135]:
# find the closest 3 schools for each property based on euclidean distance
# api has accessed limits so that it is difficult to track the route between each school and each property


school_coor = np.array(school_coor)

def get_school(i):
    # calculate euclidean distance
    distances = np.linalg.norm(school_coor-property_coor[i], axis=1)

    # find the 3 nearest school
    index = list(range(len(school_coor)))
    pairs = zip(index, distances)
    result = nsmallest(3, pairs, key=itemgetter(1))
    index_list = [i[0] for i in result]

    # store the name and the coordinates of the 3 nearest schools
    name_list = [df_school['School_Name'][i] for i in index_list]
    coor_list = [(df_school['X'][i],df_school['Y'][i])for i in index_list]

    return name_list, coor_list

# for each property, get corresponding closest schools
name_list = []
coor_list = []

for i in range(len(property_coor)):
    name_list.append(get_school(i)[0])
    coor_list.append(get_school(i)[1])

print(len(name_list))


15226


In [145]:

def get_school_ors(iter):

    flatten_list = list(chain.from_iterable(coor_list[iter*34:iter*34+34]))
    combine_coor = property_coor[iter*34:iter*34+34] + flatten_list

    matrix = client.distance_matrix(
        locations=combine_coor,
        profile='driving-car',
        metrics=['distance', 'duration'],
        validate=False,
        destinations=list(range(34,len(combine_coor))),
        sources=list(range(34)),
    )

    duration_list = []
    distance_list = []
    school_name_list = []
    school_coor_list = []

    for i in range(34):
        min_index = np.argmin(matrix['durations'][i][i*3:i*3+3])
        min_index_flat = min_index+i*3
        duration_list.append(matrix['durations'][i][min_index_flat])
        distance_list.append(matrix['distances'][i][min_index_flat])

        school_name_list.append(name_list[iter*34+i][min_index])
        school_coor_list.append(coor_list[iter*34+i][min_index])


    #print("Durations in secs: {}\n".format(matrix['durations']))
    #print("Distances in m: {}".format(matrix['distances']))

    # we only want to call the whole function once at each main iteration
    result_list = []
    result_list.append(duration_list)
    result_list.append(distance_list)
    result_list.append(school_name_list)
    result_list.append(school_coor_list)

    return result_list

# A total of 3500 routes is allowed for each call of api, so that we can pass 34 groups of property 
# with corresponding 3 closest schools at each time

duration_list = []
distance_list = []
school_name_list = []
school_coor_list = []

num_call = len(df_property)//34 + 1
for i in range(40):
    result = get_school_ors(i)
    duration_list.append(result[0])
    distance_list.append(result[1])
    school_name_list.append(result[2])
    school_coor_list.append(result[3])

print(duration_list)
print(distance_list)
print(school_name_list)
print(school_coor_list)

/Users/a123/opt/anaconda3/lib/python3.9/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 1st time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Users/a123/opt/anaconda3/lib/python3.9/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 2nd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Users/a123/opt/anaconda3/lib/python3.9/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 3rd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Users/a123/opt/anaconda3/lib/python3.9/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 4th time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,


Timeout: 

In [144]:
flatten_duration_list = list(chain.from_iterable(duration_list))
flatten_distance_list = list(chain.from_iterable(distance_list))
flatten_school_name_list = list(chain.from_iterable(school_name_list))
flatten_school_coor_list = list(chain.from_iterable(school_coor_list))

print(len(flatten_distance_list))

680
